In [1]:
import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_val, X_train = X_train[:5000], X_train[5000:]
y_val, y_train = y_train[:5000], y_train[5000:]

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_out = 10
learning_rate = 0.01

C:\Users\I863552\AppData\Local\Continuum\anaconda3\envs\mlbook\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

file_writer = tf.summary.FileWriter(log_dir("mnist_dnn"), tf.get_default_graph())

In [3]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_out, name="output")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('xentropy_loss', loss)

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar("accuracy_summary", accuracy)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [4]:
batch_size = 100
n_epochs = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for batch_index in range(len(X_train) // batch_size):
            rdm_idx = np.random.permutation(len(X_train))
            X_batch, y_batch = X_train[rdm_idx[:batch_size]], y_train[rdm_idx[:batch_size]]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if (epoch % 10 == 0):
            acc_val, acc_summary_str, loss_summary_str = sess.run([accuracy, accuracy_summary, loss_summary], feed_dict={X: X_val, y: y_val})
            file_writer.add_summary(acc_summary_str, epoch)
            file_writer.add_summary(loss_summary_str, epoch)
            print("Accuracy for validation: ", acc_val)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")
    

Accuracy for validation:  0.8756
Accuracy for validation:  0.9484
Accuracy for validation:  0.9646
Accuracy for validation:  0.9708
Accuracy for validation:  0.9734
Accuracy for validation:  0.9744
Accuracy for validation:  0.9754
Accuracy for validation:  0.977
Accuracy for validation:  0.9766
Accuracy for validation:  0.9784


In [5]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    accuracy_test = accuracy.eval(feed_dict={X:X_test, y: y_test})

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [6]:
accuracy_test

0.9799